# Basic Spectrum Generation

The first and most simple way in which TARDIS calculates spectra calculates it directly from the Monte Carlo packets after the final [Monte Carlo Iteration](../montecarlo/index.rst). This simply requires knowledge of the each packet's energy and frequency in the lab frame (see [Reference Frames](../montecarlo/propagation.rst#reference-frames)) at the end of the iteration. The only other quantity needed is the time duration of the simulation $\Delta t$, which is calculated based off of the luminosity of the supernova's photosphere (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)).

<div class="alert alert-info">

Note

The only packets which are used for this calculation are the packets which escape the outer boundary of the
computational domain -- those reabsorbed into the photosphere are not included (see [Packet Propagation](../montecarlo/propagation.rst)).

</div>

The spectrum calculation is very straightforward. A packet of energy $E_\mathrm{packet}$ contributes a
luminosity

$$L_\mathrm{packet} = \frac{E_\mathrm{packet}}{\Delta t}$$

to the spectrum at its frequency.

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.montecarlo import TARDISSpectrum
from tardis.util.base import quantity_linspace
from astropy import units as u
import numpy as np

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')

sim = Simulation.from_config(tardis_config)

sim.iterate(10000)

In [ ]:
nus = sim.runner.output_nu
nus

In [ ]:
energies = sim.runner.output_energy
energies

In [ ]:
len(nus), len(energies)

In [ ]:
luminosities = energies / sim.runner.time_of_simulation
luminosities

In [ ]:
emitted_mask = sim.runner.emitted_packet_mask
emitted_mask

In [ ]:
emitted_nus = nus[emitted_mask]
emitted_nus

In [ ]:
emitted_luminosities = luminosities[emitted_mask]
emitted_luminosities

In [ ]:
len(emitted_nus), len(emitted_luminosities)

In [ ]:
freq_start = 1.5e14 * u.Hz
freq_stop = 3e15 * u.Hz
N = 500

In [ ]:
spectrum_frequency = quantity_linspace(freq_start, freq_stop, N + 1,)
spectrum_frequency

In [ ]:
emitted_luminosity_hist = u.Quantity(np.histogram(emitted_nus,
                                                  weights=emitted_luminosities,
                                                  bins=spectrum_frequency.value,
                                                 )[0], "erg / s",)
emitted_luminosity_hist

In [ ]:
spectrum = TARDISSpectrum(spectrum_frequency, emitted_luminosity_hist)

In [ ]:
spectrum.plot(mode='frequency')

In [ ]:
spectrum.plot(mode='wavelength')